In [53]:
import pandas as pd
import numpy as np
import xml.dom.minidom

In [2]:
data = pd.read_csv("all_researchers.csv",index_col=0)

In [3]:
list(data[9:10]['institution_name'])[0] is np.NaN

True

In [16]:
import re
s = "Fall 56 sfgds- 2018"
re.search(r"(\d{4})", s).group(1)

'2018'

In [179]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))

import requests
import urllib3
import crawlera_proxies
import json


_SESSION = requests.session()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

proxies = crawlera_proxies.proxies

def get_page_content(url, retry=0):
    global _SESSION

    if retry == 5:
        return None

    response = _SESSION.get(url, proxies=proxies, verify=False)

    if response.status_code == 200:
        return response.content
    else:
        print(str(response.status_code) + " Code, waiting 10s before retrying")
        time.sleep(10)
        _SESSION = requests.Session()
        return get_page_content(url, retry+1)

def search_pubmed(query):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&retmode=json&retmax=1000&term="
    end_url = "&field=title"

    full_url = base_url + query.replace(" ", "%20") + end_url

    content = get_page_content(full_url)

    if content is None:
        return []

    x = json.loads(content)

    try:
        return x
    except KeyError:
        return []
    
def get_pubmed_pub(pubid):
    full_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id=" + pubid + "&retmode=xml"

    content = get_page_content(full_url)
    
    
    if content is None:
        return []
    

    e = xml.dom.minidom.parseString(content)
    
    
    content2 = get_page_content('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=pubmed&id=' + pubid + '&retmode=json')
    x = json.loads(content2)
    stripped_pub = list(x['result'].values())[1]
    
    
    return e, stripped_pub['pmcrefcount']

In [174]:
x = search_pubmed("(Ehab Abouheif[Author])")

In [178]:
x['esearchresult']['idlist']

['29504672',
 '28656657',
 '28521942',
 '27194576',
 '26780668',
 '26338531',
 '25963198',
 '25758336',
 '25690974',
 '25627709',
 '25228768',
 '24766229',
 '24277297',
 '23305666',
 '23225600',
 '22556252',
 '22223805',
 '21676977',
 '21552321',
 '21404423',
 '21347285',
 '21282651',
 '21282631',
 '21040424',
 '20514130',
 '20147215',
 '20083637',
 '19649305',
 '19004767',
 '18460097',
 '18184352',
 '17894383',
 '15356281',
 '12777501',
 '12114626',
 '28565215']

In [200]:
pub = get_pubmed_pub('25963198')[0]

KeyboardInterrupt: 

In [ ]:
pub.getElementsByTagName("MedlineDate")[0].firstChild.data[0:4]

In [ ]:
for pub_id in x['esearchresult']['idlist']:
    pub = get_pubmed_pub(pub_id)[0]
    try:
        abstract = pub.getElementsByTagName("AbstractText")[0].firstChild.data
    except:
        print(pub_id)

In [ ]:
raw = pub.toxml().replace('\n', '')

In [ ]:
test = xml.dom.minidom.parseString(raw)

In [ ]:
test.getElementsByTagName("PubDate")[0].getElementsByTagName("Year")[0].firstChild.data

In [5]:
import scopus
from scopus import AuthorSearch

In [7]:
scopus.load_api_key()

In [ ]:
scopus.scopu

In [10]:
x = scopus.ScopusAbstract('2-s2.0-85043317757')

{'allow': 'GET', 'Content-Encoding': 'gzip', 'Content-Type': 'text/xml;charset=UTF-8', 'Date': 'Mon, 06 Aug 2018 00:23:40 GMT', 'Server': 'Apache-Coyote/1.1', 'vary': 'Origin', 'X-ELS-APIKey': 'b689186868b1ac6fc919cf658812e778', 'X-ELS-ReqId': '64b8cfff-fa98-47da-ab7c-0a4396ea1505', 'X-ELS-ResourceVersion': 'default', 'X-ELS-Status': 'QUOTA_EXCEEDED - Quota Exceeded', 'X-ELS-TransId': '57b7e3ab-6cea-4e88-9664-6b9d01aa879f', 'X-RateLimit-Limit': '10000', 'X-RateLimit-Remaining': '0', 'X-RateLimit-Reset': '2018-08-11 09:00:37', 'Content-Length': '97', 'Connection': 'keep-alive'}


HTTPError: 429 Client Error: Too Many Requests for url: https://api.elsevier.com/content/abstract/eid/2-s2.0-85043317757?view=META_ABS

In [7]:
vars(x)

{'_affiliation_history': [<scopus.scopus_affiliation.ScopusAffiliation at 0x16e9ccc0ef0>,
 '_area_elements': [<Element 'subject-area' at 0x0000016E9CC9DE58>,
  <Element 'subject-area' at 0x0000016E9CC9DEA8>,
  <Element 'subject-area' at 0x0000016E9CC9DEF8>,
  <Element 'subject-area' at 0x0000016E9CC9DF48>,
  <Element 'subject-area' at 0x0000016E9CCA8048>,
  <Element 'subject-area' at 0x0000016E9CCA8098>,
  <Element 'subject-area' at 0x0000016E9CCA8138>,
  <Element 'subject-area' at 0x0000016E9CCA8188>,
  <Element 'subject-area' at 0x0000016E9CCA81D8>,
  <Element 'subject-area' at 0x0000016E9CCA8228>,
  <Element 'subject-area' at 0x0000016E9CCA8278>,
  <Element 'subject-area' at 0x0000016E9CCA82C8>],
 '_author_id': '7402806170',
 '_citation_count': 825,
 '_coauthor_url': 'http://api.elsevier.com/content/search/author?co-author=7402806170',
 '_current_affiliation': 'Hong Kong Polytechnic University, Department of Information Systems',
 '_date_created': (2006, 4, 15),
 '_firstname': 'Davi

In [151]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

In [152]:
import scholarly_proxy as scholarly

In [153]:
def get_author_by_scholarid(id):
    try:
        result = scholarly.Author(id).fill()
        return result
    except AttributeError:
        return None

In [154]:
x = get_author_by_scholarid('ogYgVLwAAAAJ')

In [158]:
pub = x.publications[0].fill()

In [160]:
vars(pub)

{'_filled': True,
 'bib': {'abstract': <div class="gsc_vcd_value" id="gsc_vcd_descr">Gene expression is central to the genotype-phenotype relationship in all organisms, and it is an important component of the genetic basis for evolutionary change in diverse aspects of phenotype. However, the evolution of transcriptional regulation remains understudied and poorly understood. Here we review the evolutionary dynamics of promoter, or cis-regulatory, sequences and the evolutionary mechanisms that shape them. Existing evidence indicates that populations harbor extensive genetic variation in promoter sequences, that a substantial fraction of this variation has consequences for both biochemical and organismal phenotype, and that some of this functional variation is sorted by selection. As with protein-coding sequences, rates and patterns of promoter sequence evolution differ considerably among loci and among clades for reasons that are not well understood. Studying the evolution of transcripti